### More in Depth Example on Functional API


In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [ ]:
import pandas as pd
#https://www.kaggle.com/dataset/eb9594e5b728b2eb74ff8d5e57a9b74634330bfa79d9195d6ebdc7745b9802c3?select=test_images

#HYPERPARAMETERS
BATCH_SIZE=64
WEIGHT_DECAY=0.001
LEARNING_RATE=0.001


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_images = os.getcwd() + "/train_images/" + train_df.iloc[:, 0].values
test_images = os.getcwd() + "/test_images/" + test_df.iloc[:, 0].values

train_labels = train_df.iloc[:, 1:].values
test_labels = test_df.iloc[:, 1:].values


def read_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    # In older versions you need to set shape in order to avoid error
    # on newer (2.3.0+) the following 3 lines can safely be removed
    image.set_shape((64, 64, 1))
    label[0].set_shape([])
    label[1].set_shape([])

    labels = {"first_num": label[0], "second_num": label[1]}
    return image, labels


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = (
    train_dataset.shuffle(buffer_size=len(train_labels))
    .map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = (
    test_dataset.map(read_image)
    .batch(batch_size=BATCH_SIZE)
    .prefetch(buffer_size=AUTOTUNE)
)


In [ ]:
inputs = keras.Input(shape=(64,64,1))
x =layers.Conv2D(filters=32,
                 kernel_size=(3,3),
                 padding="same",
                 kernel_regularizer= regularizers.l2(WEIGHT_DECAY)
                 )(inputs)
x=layers.BatchNormalization()(x)
x=keras.activations.relu(x)
x=layers.Conv2D(filters=64,
                 kernel_size=(3,3),
                 kernel_regularizer= regularizers.l2(WEIGHT_DECAY)
                 )(x)
x=layers.BatchNormalization()(x)
x=keras.activations.relu(x)
x=layers.MaxPooling2D()(x)
x=layers.Conv2D(filters=64,
                kernel_size=(3,3),
                activation="relu")(x)
x=layers.Conv2D(filters=128,
                kernel_size=(3,3),
                activation="relu")(x)
x=layers.MaxPooling2D()(x)
x=layers.Flatten()(x)
x=layers.Dense(128,activation="relu")(x)
x=layers.Dropout(0.5)(x)
x=layers.Dense(64, activation="relu")(x)

output1 = layers.Dense(10,name="first_num", activation="softmax")(x)
output2 = layers.Dense(10,name="second_num", activation="softmax")(x)

model=keras.Model(inputs=inputs,
                  outputs=[output1,
                           output2])

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 64, 64, 1)]  0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 64, 64, 32)   320         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_4 (BatchNor (None, 64, 64, 32)   128         conv2d_8[0][0]                   
__________________________________________________________________________________________________
tf.nn.relu_4 (TFOpLambda)       (None, 64, 64, 32)   0           batch_normalization_4[0][0]      
____________________________________________________________________________________________

In [ ]:
model.compile(loss=[keras.losses.SparseCategoricalCrossentropy(),
                    keras.losses.SparseCategoricalCrossentropy()],
              optimizer=keras.optimizers.Adam(LEARNING_RATE),
              metrics=["accuracy"])


In [ ]:
model.fit(train_dataset, epochs=5, verbose=2)
model.evaluate(test_dataset, verbose=2)

### Model Subclassing with Keras

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

In [ ]:
(x_train, y_train),(x_test, y_test)=mnist.load_data()

x_train=x_train.reshape(-1,28,28,1).astype("float32") / 255.0
x_test= x_test.reshape(-1,28,28,1).astype("float32") / 255.0


In [ ]:
# CNN => BatchNor => ReLU
# * 10

class CNNBlock(layers.Layer):
    def __init__(self, out_channels, kernel_size=3):
        super(CNNBlock, self).__init__()
        self.conv = layers.Conv2D(out_channels, kernel_size, padding="same")
        self.bn = layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv(input_tensor)
        x = self.bn(x, training=training)
        x = tf.nn.relu(x)
        return x

In [ ]:
model=keras.Sequential([
                        CNNBlock(32),
                        CNNBlock(64),
                        CNNBlock(128),
                        layers.Flatten(),
                        layers.Dense(10)
])

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
#model.build((None,28,28,3)) 
#model.summary()

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs= 3,verbose=2)

Epoch 1/3
938/938 - 697s - loss: 0.6067 - accuracy: 0.9449
Epoch 2/3
938/938 - 696s - loss: 0.0846 - accuracy: 0.9819
Epoch 3/3
938/938 - 695s - loss: 0.0381 - accuracy: 0.9890


In [ ]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 29s - loss: 0.0815 - accuracy: 0.9771


[0.08154502511024475, 0.9771000146865845]

In [ ]:
#https://www.youtube.com/watch?v=ZILIbUvp5lk&t=0s 
class ResBlock(layers.Layer):
    def __init__(self, channels):
        super(ResBlock, self).__init__()
        self.channels = channels
        self.cnn1 = CNNBlock(channels[0], 3)
        self.cnn2 = CNNBlock(channels[1], 3)
        self.cnn3 = CNNBlock(channels[2], 3)
        self.pooling = layers.MaxPooling2D()
        self.identity_mapping = layers.Conv2D(channels[1], 3, padding="same")

    def call(self, input_tensor, training=False):
        x = self.cnn1(input_tensor, training=training)
        x = self.cnn2(x, training=training)
        x = self.cnn3(x + self.identity_mapping(input_tensor), training=training,)
        x = self.pooling(x)
        return x

In [ ]:
class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    #def model(self):
        #x = keras.Input(shape=(28, 28, 1))
        #return keras.Model(inputs=[x], outputs=self.call(x))

In [ ]:
model = ResNet_Like(num_classes=10)
#model.model().summary()

In [ ]:
#model = ResNet_Like(num_classes=10)
#base_input = model.layers[0].input
#base_output = model.layers[2].output
#output = layers.Dense(10)(layers.Flatten()(base_output))
#model = keras.Model(base_input, output)
#print(model.summary())

In [ ]:
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs= 20,verbose=2)
print(model.summary())

Epoch 1/20
938/938 - 2255s - loss: 0.0930 - accuracy: 0.9715
Epoch 2/20
938/938 - 2254s - loss: 0.0361 - accuracy: 0.9887
Epoch 3/20
938/938 - 2254s - loss: 0.0304 - accuracy: 0.9904
Epoch 4/20


KeyboardInterrupt: ignored

In [ ]:
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 0s - loss: 2.3033 - accuracy: 0.1032


[2.3033447265625, 0.10320000350475311]

In [ ]:
# model.model().summary()

class ResNet_Like(keras.Model):
    def __init__(self, num_classes=10):
        super(ResNet_Like, self).__init__()
        self.block1 = ResBlock([32, 32, 64])
        self.block2 = ResBlock([128, 128, 256])
        self.block3 = ResBlock([128, 256, 512])
        self.pool = layers.GlobalAveragePooling2D()
        self.classifier = layers.Dense(num_classes)

    def call(self, input_tensor, training=False):
        x = self.block1(input_tensor, training=training)
        x = self.block2(x, training=training)
        x = self.block3(x, training=training)
        x = self.pool(x, training=training)
        x = self.classifier(x)
        return x

    def model(self):
        x = keras.Input(shape=(28, 28, 1))
        return keras.Model(inputs=[x], outputs=self.call(x))

In [ ]:
model = ResNet_Like().model()
base_input = model.layers[0].input
base_output = model.layers[2].output
output = layers.Dense(10)(layers.Flatten()(base_output))
model = keras.Model(base_input, output)

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.model().summary()

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs= 20,verbose=2)

Epoch 1/20
938/938 - 1s - loss: 0.5505 - accuracy: 0.8592
Epoch 2/20
938/938 - 1s - loss: 0.3245 - accuracy: 0.9107
Epoch 3/20
938/938 - 1s - loss: 0.2956 - accuracy: 0.9176
Epoch 4/20
938/938 - 1s - loss: 0.2828 - accuracy: 0.9211
Epoch 5/20
938/938 - 1s - loss: 0.2742 - accuracy: 0.9238
Epoch 6/20
938/938 - 1s - loss: 0.2683 - accuracy: 0.9254
Epoch 7/20
938/938 - 1s - loss: 0.2637 - accuracy: 0.9264
Epoch 8/20
938/938 - 1s - loss: 0.2602 - accuracy: 0.9275
Epoch 9/20
938/938 - 1s - loss: 0.2571 - accuracy: 0.9287
Epoch 10/20
938/938 - 1s - loss: 0.2550 - accuracy: 0.9294
Epoch 11/20
938/938 - 1s - loss: 0.2535 - accuracy: 0.9298
Epoch 12/20
938/938 - 1s - loss: 0.2512 - accuracy: 0.9308
Epoch 13/20
938/938 - 1s - loss: 0.2494 - accuracy: 0.9314
Epoch 14/20
938/938 - 1s - loss: 0.2478 - accuracy: 0.9312
Epoch 15/20
938/938 - 1s - loss: 0.2468 - accuracy: 0.9322
Epoch 16/20
938/938 - 1s - loss: 0.2455 - accuracy: 0.9322
Epoch 17/20
938/938 - 1s - loss: 0.2445 - accuracy: 0.9328
Epoch 

In [ ]:
model.save("pretrained")

### Custom Layers

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# To Avoid GPU errors
#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0


In [ ]:
x_test.shape

(10000, 784)

In [ ]:
class MyModel(keras.Model):
  def __init__(self, num_classes=10):
    super(MyModel, self).__init__()
    self.dense1=layers.Dense(64)
    self.dense2=layers.Dense(num_classes)

  def call(self, input_tensor):
      x = tf.nn.relu(self.dense1(input_tensor))
      return self.dense2(x)

In [ ]:
model=MyModel()
model.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=keras.optimizers.Adam(),
              metrics=["accuracy"])


In [ ]:
model.fit(x_train, y_train, batch_size=64 , epochs= 20,verbose=2)

Epoch 1/20
938/938 - 2s - loss: 0.3554 - accuracy: 0.9014
Epoch 2/20
938/938 - 2s - loss: 0.1739 - accuracy: 0.9499
Epoch 3/20
938/938 - 2s - loss: 0.1280 - accuracy: 0.9627
Epoch 4/20
938/938 - 2s - loss: 0.1006 - accuracy: 0.9705
Epoch 5/20
938/938 - 2s - loss: 0.0831 - accuracy: 0.9758
Epoch 6/20
938/938 - 2s - loss: 0.0698 - accuracy: 0.9799
Epoch 7/20
938/938 - 1s - loss: 0.0611 - accuracy: 0.9814
Epoch 8/20
938/938 - 2s - loss: 0.0523 - accuracy: 0.9845
Epoch 9/20
938/938 - 1s - loss: 0.0451 - accuracy: 0.9867
Epoch 10/20
938/938 - 1s - loss: 0.0403 - accuracy: 0.9881
Epoch 11/20
938/938 - 2s - loss: 0.0352 - accuracy: 0.9899
Epoch 12/20
938/938 - 2s - loss: 0.0316 - accuracy: 0.9907
Epoch 13/20
938/938 - 2s - loss: 0.0267 - accuracy: 0.9924
Epoch 14/20
938/938 - 2s - loss: 0.0239 - accuracy: 0.9933
Epoch 15/20
938/938 - 1s - loss: 0.0215 - accuracy: 0.9943
Epoch 16/20
938/938 - 2s - loss: 0.0187 - accuracy: 0.9949
Epoch 17/20
938/938 - 2s - loss: 0.0167 - accuracy: 0.9955
Epoch 

In [ ]:
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

313/313 - 0s - loss: 0.0964 - accuracy: 0.9754


[0.09635300189256668, 0.9753999710083008]

In [ ]:
class Dense(layers.Layer):
    def __init__(self, units, input_dim):
        super(Dense, self).__init__()
        self.w = self.add_weight(
            name="w",
            shape=(input_dim, units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b", shape=(units,), initializer="zeros", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In [ ]:
class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64,784)
        self.dense2 = Dense(num_classes,64)

        # self.dense1 = layers.Dense(64)
        # self.dense3 = layers.Dense(num_classes)

    def call(self, x):
        x = tf.nn.relu(self.dense1(x))
        return self.dense2(x)

In [ ]:

model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)

Epoch 1/2
938/938 - 2s - loss: 0.4071 - accuracy: 0.8904
Epoch 2/2
938/938 - 2s - loss: 0.1949 - accuracy: 0.9446


In [ ]:
class MyRelu(layers.Layer):
  def __init__(self):
    super(MyRelu, self).__init__()
  def call(self,x):
    return tf.math.maximum(x,0)


In [ ]:
class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64,784)
        self.dense2 = Dense(num_classes,64)
        self.relu =MyRelu()

        # self.dense1 = layers.Dense(64)
        # self.dense3 = layers.Dense(num_classes)

    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)

In [ ]:
model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)

Epoch 1/2
938/938 - 2s - loss: 0.4064 - accuracy: 0.8934
Epoch 2/2
938/938 - 2s - loss: 0.1972 - accuracy: 0.9438


In [ ]:

class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            name="w",
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            name="b", shape=(self.units,), initializer="random_normal", trainable=True,
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [ ]:

class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()

    def call(self, x):
        return tf.math.maximum(x, 0)


class MyModel(keras.Model):  # model.fit, model.evalute, model.predict
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()

        # self.dense1 = layers.Dense(64)
        # self.dense3 = layers.Dense(num_classes)

    def call(self, x):
        x = self.relu(self.dense1(x))
        return self.dense2(x)


model = MyModel()
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"],
)

model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=32, verbose=2)

Epoch 1/2
1875/1875 - 2s - loss: 0.3448 - accuracy: 0.9051
Epoch 2/2
1875/1875 - 2s - loss: 0.1638 - accuracy: 0.9517
313/313 - 0s - loss: 0.1371 - accuracy: 0.9591


[0.13708026707172394, 0.9591000080108643]

### Saving and loading models

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# To Avoid GPU errors
#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [ ]:
model1 =keras.Sequential([
                          layers.Dense(64, activation="relu"),
                          layers.Dense(10)
])

inputs=keras.Input(784)
x=layers.Dense(64,activation="relu")(inputs)
outputs=layers.Dense(10,activation="softmax")(x)
model2=keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
class MyModel(keras.Model):
  def __init__(self,):
    super(MyModel, self).__init__()
    self.dense1 = layers.Dense(64, activation="relu")
    self.dense2= layers.Dense(10)

  def call(self, input_tensor):
    x=tf.nn.relu(self.dense1(input_tensor))
    return self.dense2(x)

In [ ]:
model3 = MyModel()

In [ ]:
#Save and load model weights


In [ ]:
model = model1
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)
model.save_weights("saved_model/")

Epoch 1/2
938/938 - 2s - loss: 0.1261 - accuracy: 0.9627
Epoch 2/2
938/938 - 1s - loss: 0.0984 - accuracy: 0.9713
157/157 - 0s - loss: 0.1044 - accuracy: 0.9680


In [ ]:
model.load_weights("saved_model/")
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=64, epochs=2, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/2
938/938 - 2s - loss: 0.0828 - accuracy: 0.9750
Epoch 2/2
938/938 - 1s - loss: 0.0687 - accuracy: 0.9794
157/157 - 0s - loss: 0.0916 - accuracy: 0.9731


[0.09163733571767807, 0.9731000065803528]

In [ ]:
#model.save_weights("saved_model/", save_format="h5")

In [ ]:
# Save and load entire model (Serializing model):
# - Save weights 
# - Save model architecture
# - Training configuration
# - optimizer and state 



In [ ]:
model =model1
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)
model.save("complete_saved_model/")

Epoch 1/10
938/938 - 2s - loss: 0.0604 - accuracy: 0.9819
Epoch 2/10
938/938 - 1s - loss: 0.0518 - accuracy: 0.9846
Epoch 3/10
938/938 - 1s - loss: 0.0457 - accuracy: 0.9866
Epoch 4/10
938/938 - 1s - loss: 0.0402 - accuracy: 0.9879
Epoch 5/10
938/938 - 1s - loss: 0.0356 - accuracy: 0.9895
Epoch 6/10
938/938 - 1s - loss: 0.0308 - accuracy: 0.9913
Epoch 7/10
938/938 - 1s - loss: 0.0276 - accuracy: 0.9920
Epoch 8/10
938/938 - 1s - loss: 0.0241 - accuracy: 0.9932
Epoch 9/10
938/938 - 1s - loss: 0.0216 - accuracy: 0.9939
Epoch 10/10
938/938 - 1s - loss: 0.0190 - accuracy: 0.9950
157/157 - 0s - loss: 0.0891 - accuracy: 0.9757
INFO:tensorflow:Assets written to: complete_saved_model/assets


In [ ]:
model= keras.models.load_model("complete_saved_model/")

In [ ]:
model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/10
938/938 - 2s - loss: 0.0187 - accuracy: 0.9943
Epoch 2/10
938/938 - 1s - loss: 0.0163 - accuracy: 0.9956
Epoch 3/10
938/938 - 1s - loss: 0.0136 - accuracy: 0.9968
Epoch 4/10
938/938 - 1s - loss: 0.0117 - accuracy: 0.9971
Epoch 5/10
938/938 - 1s - loss: 0.0115 - accuracy: 0.9970
Epoch 6/10
938/938 - 1s - loss: 0.0105 - accuracy: 0.9972
Epoch 7/10
938/938 - 1s - loss: 0.0079 - accuracy: 0.9984
Epoch 8/10
938/938 - 1s - loss: 0.0092 - accuracy: 0.9974
Epoch 9/10
938/938 - 1s - loss: 0.0064 - accuracy: 0.9986
Epoch 10/10
938/938 - 1s - loss: 0.0073 - accuracy: 0.9983
157/157 - 0s - loss: 0.1020 - accuracy: 0.9755


[0.1019556000828743, 0.9754999876022339]

###  Transfer Learning, Fine Tuning and TensorFlow Hub

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

# To Avoid GPU errors
#physical_devices = tf.config.list_physical_devices("GPU")
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28 * 28).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28 * 28).astype("float32") / 255.0

In [ ]:
#pretrained model 
model= keras.models.load_model("complete_saved_model/")
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_44 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_45 (Dense)             (None, 10)                650       
Total params: 50,890
Trainable params: 50,890
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# freeze all model layers
model.trainable = False 

for layer in model.layers:
  assert layer.trainable == False
  layer.trainable = False

In [ ]:
from keras.applications.inception_v3 import InceptionV3

In [ ]:
#pretrained keras model 
x =tf.random.normal(shape=(5,299,299,3))
y= tf.constant([0,1,2,3,4])

model = keras.applications.InceptionV3(include_top=True)
print(model.summary())

96116736/96112376 [==============================] - 1s 0us/step
Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
______________________

In [ ]:
base_inputs=model.layers[0].input 
base_outputs=model.layers[-2].output 
final_outputs = layers.Dense(5)(base_outputs)
new_model=keras.Model(inputs=base_inputs, outputs=final_outputs)

new_model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"])

new_model.fit(x, y, epochs=10, verbose=2)
#overfitting

In [ ]:
# pretrained Hub model

  !pip install --upgrade tensorflow_hub

  import tensorflow_hub as hub

Requirement already up-to-date: tensorflow_hub in /usr/local/lib/python3.7/dist-packages (0.12.0)


In [ ]:
# pretrained Hub model
x =tf.random.normal(shape=(5,299,299,3))
y= tf.constant([0,1,2,3,4])
url = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4" 

base_model=hub.KerasLayer(url, input_shape=(299,299,3))
base_model.trainable=False
model =keras.Sequential([
                         base_model,
                         layers.Dense(128, activation="relu"),
                         layers.Dense(64, activation='relu'),
                         layers.Dense(5)
])



In [ ]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dense_55 (Dense)             (None, 128)               262272    
_________________________________________________________________
dense_56 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_57 (Dense)             (None, 5)                 325       
Total params: 22,073,637
Trainable params: 270,853
Non-trainable params: 21,802,784
_________________________________________________________________
None


In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(),
    metrics=["accuracy"])

model.fit(x, y, epochs=3, verbose=2)
#overfitting

Epoch 1/3
1/1 - 1s - loss: 0.6959 - accuracy: 1.0000
Epoch 2/3
1/1 - 0s - loss: 0.6323 - accuracy: 1.0000
Epoch 3/3
1/1 - 0s - loss: 0.5654 - accuracy: 1.0000


### Tensorflow Datasets

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [ ]:
(ds_train, ds_test), ds_info = tfds.load("mnist",split=["train", "test"],shuffle_files=True,as_supervised=True,  with_info=True)

# fig = tfds.show_examples(ds_train, ds_info, rows=4, cols=4)
# print(ds_info)
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [24]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    "imdb_reviews",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # tuple (img, label) otherwise dict
    with_info=True,  # info about dataset
)


In [ ]:
tokenizer = tfds.features.text.Tokenizer()

In [ ]:
def build_vocabulary():
    vocabulary = set()
    for text, _ in ds_train:
        vocabulary.update(tokenizer.tokenize(text.numpy().lower()))
    return vocabulary


vocabulary = build_vocabulary()

encoder = tfds.features.text.TokenTextEncoder(
    list(vocabulary), oov_token="<UNK>", lowercase=True, tokenizer=tokenizer)



In [ ]:
def my_enc(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

In [ ]:
# py_function does not set the shape of the returned tensors.
def encode_map_fn(text, label):
    encoded_text, label = tf.py_function(
        my_enc, inp=[text, label], Tout=(tf.int64, tf.int64))
    # `tf.data.Datasets` work best if all components have a shape set
    #  so set the shapes manually:
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
ds_train = ds_train.map(encode_map_fn, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(1000)
ds_train = ds_train.padded_batch(32, padded_shapes=([None], ()))
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(encode_map_fn)
ds_test = ds_test.padded_batch(32, padded_shapes=([None], ()))

### Data Augmentation


In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds


In [11]:

(ds_train, ds_test), ds_info = tfds.load("cifar10",split=["train", "test"],shuffle_files=True, 
                                         as_supervised=True,  # return tuple (img, label), si non a dict
                                         with_info=True,  # info about dataset
)



In [15]:
def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label

In [16]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32


def augment(image, label):
    new_height = new_width = 32
    image = tf.image.resize(image, (new_height, new_width))

    if tf.random.uniform((), minval=0, maxval=1) < 0.1:
        image = tf.tile(tf.image.rgb_to_grayscale(image), [1, 1, 3])

    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2)

    image = tf.image.random_flip_left_right(image)  # 50%
    # image = tf.image.random_flip_up_down(image) #%50%

    return image, label

In [17]:
# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
# ds_train = ds_train.map(augment)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(BATCH_SIZE)
ds_test = ds_train.prefetch(AUTOTUNE)

In [18]:
# TF >= 2.3.0
data_augmentation = keras.Sequential([layers.experimental.preprocessing.Resizing(height=32, width=32,),
                                      layers.experimental.preprocessing.RandomFlip(mode="horizontal"),
                                      layers.experimental.preprocessing.RandomContrast(factor=0.1,)])


In [19]:

model = keras.Sequential([keras.Input((32, 32, 3)),
                          data_augmentation,
                          layers.Conv2D(4, 3, padding="same", activation="relu"),
                          layers.Conv2D(8, 3, padding="same", activation="relu"),
                          layers.MaxPooling2D(),
                          layers.Conv2D(16, 3, padding="same", activation="relu"),
                          layers.Flatten(),
                          layers.Dense(64, activation="relu"),
                          layers.Dense(10)])





In [20]:
model.compile(
    optimizer=keras.optimizers.Adam(3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [21]:
model.fit(ds_train, epochs=5, verbose=2)

Epoch 1/5
1563/1563 - 12s - loss: 1.7781 - accuracy: 0.3620
Epoch 2/5
1563/1563 - 4s - loss: 1.3968 - accuracy: 0.5038
Epoch 3/5
1563/1563 - 4s - loss: 1.2670 - accuracy: 0.5523
Epoch 4/5
1563/1563 - 4s - loss: 1.1899 - accuracy: 0.5832
Epoch 5/5
1563/1563 - 4s - loss: 1.1314 - accuracy: 0.6013


In [22]:
model.evaluate(ds_test)
# We use 5 epochs, just for learning purpose, 
# Surely we should do more fine tuning to increase accuracy

1563/1563 [==============================] - 4s 2ms/step - loss: 1.0663 - accuracy: 0.6269


[1.0662678480148315, 0.6268600225448608]

### Callbacks with Keras and Writing Custom Callbacks 


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

# fig = tfds.show_examples(ds_train, ds_info, rows=4, cols=4)
# print(ds_info)


def normalize_img(image, label):
    """Normalizes images"""
    return tf.cast(image, tf.float32) / 255.0, label


AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

# Setup for train dataset
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

# Setup for test Dataset
ds_test = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_train.batch(128)
ds_test = ds_train.prefetch(AUTOTUNE)


local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:

model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)


In [ ]:
save_callback = keras.callbacks.ModelCheckpoint("checkpoint/",
                                                save_weights_only=True,
                                                monitor="accuracy",
                                                save_best_only= False)

In [ ]:
def scheduler(epoch, lr):
  if epoch < 2:
    return lr
  else :
    return lr * 0.99

lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler,
                                                     verbose=1)

In [ ]:
#https://www.tensorflow.org/guide/keras/custom_callback

class Customer_Callback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))


In [ ]:
#https://www.tensorflow.org/guide/keras/custom_callback

class Customer_Callback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get('accuracy')> 0.9:
      print("Accuracy over 90%, quitting training")
      self.model.stop_training = True

In [ ]:

model.compile(
    optimizer=keras.optimizers.Adam(0.001),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

model.fit(ds_train,
          epochs=5,
          verbose=2,
          callbacks=[save_callback,lr_scheduler,Customer_Callback()])
model.evaluate(ds_test)



Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.0010000000474974513.
469/469 - 1s - loss: 0.0031 - accuracy: 0.9993
Accuracy over 90%, quitting training
469/469 [==============================] - 1s 2ms/step - loss: 0.0018 - accuracy: 0.9999


[0.0017512183403596282, 0.9998999834060669]

### Customizing Model.Fit

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

11493376/11490434 [==============================] - 0s 0us/step


In [2]:
model = keras.Sequential(
    [
        layers.Input(shape=(28, 28, 1)),
        layers.Conv2D(64, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Conv2D(128, (3, 3), padding="same"),
        layers.ReLU(),
        layers.Flatten(),
        layers.Dense(10),
    ],
    name="model",
    )

In [3]:
class CustomFit(keras.Model):
    def __init__(self, model):
        super(CustomFit, self).__init__()
        self.model = model

    def compile(self, optimizer, loss):
        super(CustomFit, self).compile()
        self.optimizer = optimizer
        self.loss = loss

    def train_step(self, data):
        x, y = data

        with tf.GradientTape() as tape:
            # Caclulate predictions
            y_pred = self.model(x, training=True)

            # Loss
            loss = self.loss(y, y_pred)

        # Gradients
        training_vars = self.trainable_variables
        gradients = tape.gradient(loss, training_vars)

        # Optimizer
        self.optimizer.apply_gradients(zip(gradients, training_vars))
        acc_metric.update_state(y, y_pred)

        return {"loss": loss, "accuracy": acc_metric.result()}

    def test_step(self, data):
        x, y = data

        # Compute predictions
        y_pred = self.model(x, training=False)

        # Updates the metrics tracking the loss
        loss = self.loss(y, y_pred)

        # Update the metrics.
        acc_metric.update_state(y, y_pred)
        return {"loss": loss, "accuracy": acc_metric.result()}

In [5]:
acc_metric = keras.metrics.SparseCategoricalAccuracy(name="accuracy")


In [6]:
training = CustomFit(model)


In [7]:
training.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [8]:
training.fit(x_train, y_train, batch_size=64, epochs=2)
training.evaluate(x_test, y_test, batch_size=64)

Epoch 1/2
938/938 [==============================] - 8s 8ms/step - loss: 0.0416 - accuracy: 0.9873
Epoch 2/2
157/157 [==============================] - 1s 4ms/step - loss: 0.0374 - accuracy: 0.9891


[0.9890769124031067, 0.00026514354976825416]